In [27]:
import torch
from datasets import load_dataset

from transformers import AutoTokenizer
from transformers import T5ForConditionalGeneration, T5Tokenizer

In [17]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [34]:
train_data = load_dataset('wikisql', split='train+validation')
test_data = load_dataset('wikisql', split='test')

In [38]:
test_data[0]
# dict_keys(['phase', 'question', 'table', 'sql'])

{'phase': 1,
 'question': "What is terrence ross' nationality",
 'table': {'header': ['Player',
   'No.',
   'Nationality',
   'Position',
   'Years in Toronto',
   'School/Club Team'],
  'page_title': 'Toronto Raptors all-time roster',
  'page_id': '',
  'types': ['text', 'text', 'text', 'text', 'text', 'text'],
  'id': '1-10015132-16',
  'section_title': 'R',
  'caption': 'R',
  'rows': [['Aleksandar Radojević',
    '25',
    'Serbia',
    'Center',
    '1999-2000',
    'Barton CC (KS)'],
   ['Shawn Respert',
    '31',
    'United States',
    'Guard',
    '1997-98',
    'Michigan State'],
   ['Quentin Richardson',
    'N/A',
    'United States',
    'Forward',
    '2013-present',
    'DePaul'],
   ['Alvin Robertson',
    '7, 21',
    'United States',
    'Guard',
    '1995-96',
    'Arkansas'],
   ['Carlos Rogers',
    '33, 34',
    'United States',
    'Forward-Center',
    '1995-98',
    'Tennessee State'],
   ['Roy Rogers', '9', 'United States', 'Forward', '1998', 'Alabama'],
   

In [19]:
type(train_data), type(test_data)

(datasets.arrow_dataset.Dataset, datasets.arrow_dataset.Dataset)

In [20]:
def format_dataset(example):
    return {'input': 'translate to SQL: ' + example['question'], 'target': example['sql']['human_readable']}

In [25]:
train_data = train_data.map(format_dataset, remove_columns=train_data.column_names)
test_data = test_data.map(format_dataset, remove_columns=test_data.column_names)

Map:   0%|          | 0/64776 [00:00<?, ? examples/s]

Map:   0%|          | 0/15878 [00:00<?, ? examples/s]

({'input': 'translate to SQL: Tell me what the notes are for South Australia ',
  'target': 'SELECT Notes FROM table WHERE Current slogan = SOUTH AUSTRALIA'},
 {'input': "translate to SQL: What is terrence ross' nationality",
  'target': 'SELECT Nationality FROM table WHERE Player = Terrence Ross'})

In [28]:
tokenizer = AutoTokenizer.from_pretrained('t5-small')

loading file spiece.model from cache at /home/rmaurya/.cache/huggingface/hub/models--t5-small/snapshots/df1b051c49625cf57a3d0d8d3863ed4d13564fe4/spiece.model
loading file tokenizer.json from cache at /home/rmaurya/.cache/huggingface/hub/models--t5-small/snapshots/df1b051c49625cf57a3d0d8d3863ed4d13564fe4/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /home/rmaurya/.cache/huggingface/hub/models--t5-small/snapshots/df1b051c49625cf57a3d0d8d3863ed4d13564fe4/tokenizer_config.json
loading file chat_template.jinja from cache at None


In [32]:
def map_to_length(x):
    print(x)
    x['input_len'] = len(tokenizer(x['input'].input_ids))
    x["input_longer_256"] = int(x["input_len"] > 256)
    x["input_longer_128"] = int(x["input_len"] > 128)
    x["input_longer_64"] = int(x["input_len"] > 64)
    x["out_len"] = len(tokenizer(x["target"]).input_ids)
    x["out_longer_256"] = int(x["out_len"] > 256)
    x["out_longer_128"] = int(x["out_len"] > 128)
    x["out_longer_64"] = int(x["out_len"] > 64)
    return x

In [33]:
sample_size = 10000
data_stats = train_data.select(range(sample_size)).map(map_to_length, num_proc=4)

Map (num_proc=4):   0%|          | 0/10000 [00:00<?, ? examples/s]

{'input': 'translate to SQL: What kind of vehicle is the agra 1050?', 'target': 'SELECT Vehicle Category FROM table WHERE Till Agra = 1050'}
{'input': 'translate to SQL: What year was incumbent mark w. hannaford elected?', 'target': 'SELECT MIN First elected FROM table WHERE Incumbent = Mark W. Hannaford'}{'input': 'translate to SQL: Tell me what the notes are for South Australia ', 'target': 'SELECT Notes FROM table WHERE Current slogan = SOUTH AUSTRALIA'}

{'input': 'translate to SQL: How many times is a score for stolen ends recorded for France?', 'target': 'SELECT COUNT Stolen Ends FROM table WHERE Country = France'}


AttributeError: 'str' object has no attribute 'input_ids'